In [1]:
import os
import numpy as np
from shutil import copyfile
from tqdm.notebook import tqdm
import pandas as pd
import cv2

np.random.seed(0)

In [2]:
languages = ['Tamil', 'Hindi', 'Telugu', 'Malayalam', 'Punjabi']
splits = {'Train':10000, 'Val':200, 'Test':200}
img_w_new, img_h_new = 320, 320

In [3]:
# Train
for split, split_count in splits.items():
    print(f'Processing {split} split')
    dest_img_dir = f'Data/Detection-V2-Big/{split}/Images'
    dest_ann_dir_quad = f'Data/Detection-V2-Big/{split}/Annotations-QUAD'
    dest_ann_dir_aabb = f'Data/Detection-V2-Big/{split}/Annotations-AABB'
    os.makedirs(dest_img_dir)
    os.makedirs(dest_ann_dir_quad)
    os.makedirs(dest_ann_dir_aabb)

    for language in tqdm(languages, desc='L1'):
        src_img_dir = f'Data/Synthetic-{language}/Image/' + '{}'
        src_ann_dir = f'Data/Synthetic-{language}/Annotation/' + '{}'
    
        if split == 'Train':
            sub_dir_range = range(1, 20+1) 
        elif split == 'Val':
            sub_dir_range = range(21, 22+1) 
        elif split == 'Test':
            sub_dir_range = range(23, 24+1) 
        size = split_count//len(sub_dir_range)
        for i in tqdm(sub_dir_range, leave=False, desc='L2'):
            src_img_dir_ = src_img_dir.format(i)
            src_ann_dir_ = src_ann_dir.format(i)
            img_fns = os.listdir(src_img_dir_)
            img_fns = np.random.choice(img_fns, size=size, replace=False)

            for img_fn in tqdm(img_fns, leave=False, desc='L3'):

                img_fn_new = f'{language}_{i}_{img_fn}'
                src_img_fp = os.path.join(src_img_dir_, img_fn)
                dest_img_fp = os.path.join(dest_img_dir, img_fn_new)
                img = cv2.imread(src_img_fp)
                img_h, img_w, img_c = img.shape 
                img = cv2.resize(img, (img_w_new, img_h_new))
                cv2.imwrite(dest_img_fp, img)

                ann_fn = f'{img_fn.split(".")[0]}.txt'
                ann_fn_new = f'{language}_{i}_{ann_fn.replace("txt", "csv")}'
                src_ann_fp = os.path.join(src_ann_dir_, ann_fn)
                ann_df_quad = pd.read_csv(src_ann_fp, sep=' ', header=None)
                ann_df_quad.columns = ['x1', 'x2', 'x3', 'x4', 'y1', 'y2', 'y3', 'y4', 'text']
                w_ratio = img_w_new / img_w
                h_ratio = img_h_new / img_h
                ann_df_quad[['x1', 'x2', 'x3', 'x4']] = ann_df_quad[['x1', 'x2', 'x3', 'x4']] * w_ratio
                ann_df_quad[['y1', 'y2', 'y3', 'y4']] = ann_df_quad[['y1', 'y2', 'y3', 'y4']] * h_ratio
                ann_df_quad = ann_df_quad[['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']]
                ann_df_quad = ann_df_quad.clip(lower=0).astype('int32')
                dest_ann_fp_quad = os.path.join(dest_ann_dir_quad, ann_fn_new)
                ann_df_quad.to_csv(dest_ann_fp_quad, index=False, header=True)

                ann_df_aabb = []
                for row_id, row in ann_df_quad.iterrows():
                    coords = row.values.reshape(4, 1, 2).astype(np.float32)
                    x, y, w, h = cv2.boundingRect(coords)
                    ann_df_aabb.append([x, y, w, h])
                ann_df_aabb = pd.DataFrame(ann_df_aabb, columns=['x', 'y', 'w', 'h'])
                dest_ann_fp_aabb = os.path.join(dest_ann_dir_aabb, ann_fn_new)
                ann_df_aabb.to_csv(dest_ann_fp_aabb, index=False, header=True)

Processing Train split



Processing Val split



Processing Test split


# English Detection Dataset

Data Source: https://rrc.cvc.uab.es/?ch=4&com=downloads

In [2]:
img_w_new, img_h_new = 320, 320
train_image_dir = 'Data/ICDAR-2015-Incidental-Scene-Text/Train-Images'
train_ann_dir = 'Data/ICDAR-2015-Incidental-Scene-Text/Train-Anns'
test_image_dir = 'Data/ICDAR-2015-Incidental-Scene-Text/Test-Images'
test_ann_dir = 'Data/ICDAR-2015-Incidental-Scene-Text/Test-Anns'

In [3]:
dest_img_dir = f'Data/Detection-English/Train/Images'
dest_ann_dir_quad = f'Data/Detection-English/Train/Annotations-QUAD'
dest_ann_dir_aabb = f'Data/Detection-English/Train/Annotations-AABB'
os.makedirs(dest_img_dir)
os.makedirs(dest_ann_dir_quad)
os.makedirs(dest_ann_dir_aabb)

img_fns = os.listdir(train_image_dir)
img_fns = np.random.choice(img_fns, size=1000, replace=False)

for img_fn in tqdm(img_fns, leave=False, desc='L3'):

    img_fn_new = f'English_{img_fn}'
    src_img_fp = os.path.join(train_image_dir, img_fn)
    dest_img_fp = os.path.join(dest_img_dir, img_fn_new)
    img = cv2.imread(src_img_fp)
    img_h, img_w, img_c = img.shape 
    img = cv2.resize(img, (img_w_new, img_h_new))
    cv2.imwrite(dest_img_fp, img)

    ann_fn = f'gt_{img_fn.split(".")[0]}.txt'
    ann_fn_new = img_fn_new.replace("jpg", "csv")
    src_ann_fp = os.path.join(train_ann_dir, ann_fn)
    with open(src_ann_fp, 'r', encoding='utf-8-sig') as file:
        lines = [line.strip(',').split(',')[:8] for line in file.readlines()]
        ann_df_quad = pd.DataFrame(lines).astype('int64')
        ann_df_quad.columns = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']
    
    w_ratio = img_w_new / img_w
    h_ratio = img_h_new / img_h
    ann_df_quad[['x1', 'x2', 'x3', 'x4']] = ann_df_quad[['x1', 'x2', 'x3', 'x4']] * w_ratio
    ann_df_quad[['y1', 'y2', 'y3', 'y4']] = ann_df_quad[['y1', 'y2', 'y3', 'y4']] * h_ratio
    ann_df_quad = ann_df_quad[['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']]
    ann_df_quad = ann_df_quad.clip(lower=0).astype('int32')
    dest_ann_fp_quad = os.path.join(dest_ann_dir_quad, ann_fn_new)
    ann_df_quad.to_csv(dest_ann_fp_quad, index=False, header=True)

    ann_df_aabb = []
    for row_id, row in ann_df_quad.iterrows():
        coords = row.values.reshape(4, 1, 2).astype(np.float32)
        x, y, w, h = cv2.boundingRect(coords)
        ann_df_aabb.append([x, y, w, h])
    ann_df_aabb = pd.DataFrame(ann_df_aabb, columns=['x', 'y', 'w', 'h'])
    dest_ann_fp_aabb = os.path.join(dest_ann_dir_aabb, ann_fn_new)
    ann_df_aabb.to_csv(dest_ann_fp_aabb, index=False, header=True)

In [4]:
dest_img_dir_val = f'Data/Detection-English/Val/Images'
dest_ann_dir_quad_val = f'Data/Detection-English/Val/Annotations-QUAD'
dest_ann_dir_aabb_val = f'Data/Detection-English/Val/Annotations-AABB'
dest_img_dir_test = f'Data/Detection-English/Test/Images'
dest_ann_dir_quad_test = f'Data/Detection-English/Test/Annotations-QUAD'
dest_ann_dir_aabb_test = f'Data/Detection-English/Test/Annotations-AABB'
os.makedirs(dest_img_dir_val)
os.makedirs(dest_ann_dir_quad_val)
os.makedirs(dest_ann_dir_aabb_val)
os.makedirs(dest_img_dir_test)
os.makedirs(dest_ann_dir_quad_test)
os.makedirs(dest_ann_dir_aabb_test)

img_fns = os.listdir(test_image_dir)
img_fns = np.random.choice(img_fns, size=400, replace=False)

for i, img_fn in tqdm(enumerate(img_fns), leave=False, desc='L3', total=len(img_fns)):

    img_fn_new = f'English_{img_fn}'
    src_img_fp = os.path.join(test_image_dir, img_fn)
    if i < 200:
        dest_img_fp = os.path.join(dest_img_dir_val, img_fn_new)
    else:
        dest_img_fp = os.path.join(dest_img_dir_test, img_fn_new)
    img = cv2.imread(src_img_fp)
    img_h, img_w, img_c = img.shape 
    img = cv2.resize(img, (img_w_new, img_h_new))
    cv2.imwrite(dest_img_fp, img)

    ann_fn = f'gt_{img_fn.split(".")[0]}.txt'
    ann_fn_new = img_fn_new.replace("jpg", "csv")
    src_ann_fp = os.path.join(test_ann_dir, ann_fn)
    with open(src_ann_fp, 'r', encoding='utf-8-sig') as file:
        lines = [line.strip(',').split(',')[:8] for line in file.readlines()]
        ann_df_quad = pd.DataFrame(lines).astype('int64')
        ann_df_quad.columns = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']
    
    w_ratio = img_w_new / img_w
    h_ratio = img_h_new / img_h
    ann_df_quad[['x1', 'x2', 'x3', 'x4']] = ann_df_quad[['x1', 'x2', 'x3', 'x4']] * w_ratio
    ann_df_quad[['y1', 'y2', 'y3', 'y4']] = ann_df_quad[['y1', 'y2', 'y3', 'y4']] * h_ratio
    ann_df_quad = ann_df_quad[['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']]
    ann_df_quad = ann_df_quad.clip(lower=0).astype('int32')
    if i < 200:
        dest_ann_fp_quad = os.path.join(dest_ann_dir_quad_val, ann_fn_new)
    else:
        dest_ann_fp_quad = os.path.join(dest_ann_dir_quad_test, ann_fn_new)
    ann_df_quad.to_csv(dest_ann_fp_quad, index=False, header=True)

    ann_df_aabb = []
    for row_id, row in ann_df_quad.iterrows():
        coords = row.values.reshape(4, 1, 2).astype(np.float32)
        x, y, w, h = cv2.boundingRect(coords)
        ann_df_aabb.append([x, y, w, h])
    ann_df_aabb = pd.DataFrame(ann_df_aabb, columns=['x', 'y', 'w', 'h'])
    if i < 200:
        dest_ann_fp_aabb = os.path.join(dest_ann_dir_aabb_val, ann_fn_new)
    else:
        dest_ann_fp_aabb = os.path.join(dest_ann_dir_aabb_test, ann_fn_new)
    ann_df_aabb.to_csv(dest_ann_fp_aabb, index=False, header=True)